## 과제 Book 클래스

In [1]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

people = [
    Person('Tom', 10),
    Person('Stve', 12),
    Person('John', 8),
]

In [2]:
first_people = people[0]

In [3]:
first_people.name

'Tom'

In [4]:
first_people.age

10

In [5]:
getattr(first_people, 'name')

'Tom'

In [6]:
getattr(first_people, 'age')

10

In [9]:
# getattr
print(first_people.name)
print(getattr(first_people, 'name'))

# setattr
# first_people.age = 20
setattr(first_people, 'age', 20)
print(first_people.age)

# hasattr
print(hasattr(first_people, 'age'))
print(hasattr(first_people, 'region'))

Tom
Tom
20
True
False


In [10]:
def sortfn(list, standard_name):
    return sorted(list, key = lambda i: getattr(i, standard_name))

In [13]:
for person in people:
    print(person.name, person.age)

Tom 20
Stve 12
John 8


In [15]:
for person in sortfn(people, 'age'):
    print(person.name, person.age)

John 8
Stve 12
Tom 20


## 인자를 받는 장식자 base

In [23]:
def base_10(fn):
    def wrap(x, y):
        print("called base_10")
        return fn(x, y) + 10
    return wrap

def base_20(fn):
    def wrap(x, y):
        print("called base_20")
        return fn(x, y) + 20
    return wrap

def base_30(fn):
    def wrap(x, y):
        print("called base_30")
        return fn(x, y) + 30
    return wrap

@base_10
@base_20
@base_30
def mysum(x, y):
    return x + y

# mysum = base_30(mysum)
# mysum = base_20(mysum)
# mysum = base_10(mysum)

print(mysum(1, 2))

called base_10
called base_20
called base_30
63
